In [1]:
! pip install strsimpy  
! pip install pyspellchecker  

     |████████████████████████████████| 51kB 2.3MB/s 
     |████████████████████████████████| 1.9MB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np
import random
import re
import gensim
from gensim.models import Word2Vec
from strsimpy.jaro_winkler import JaroWinkler
import nltk
from nltk.corpus import stopwords
from spellchecker import SpellChecker
import itertools

##### Libraries
! pip install pyspellchecker
##### Models :
https://github.com/eyaler/word2vec-slim/blob/master/GoogleNews-vectors-negative300-SLIM.bin.gz  
https://github.com/mmihaltz/word2vec-GoogleNews-vectors/blob/master/GoogleNews-vectors-negative300.bin.gz

In [3]:
!wget -P ./data -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-01-13 20:06:27--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.238.181
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.238.181|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘./data/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  53.1MB/s    in 28s     

2020-01-13 20:06:55 (56.1 MB/s) - ‘./data/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [4]:
pathword2vec = './data/GoogleNews-vectors-negative300.bin.gz'
model_w2v = gensim.models.KeyedVectors.load_word2vec_format(pathword2vec,
                                                            binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [5]:
nltk.download('stopwords')
voc_stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
df_aircraft = pd.read_csv('./data/df_aircraft.csv', sep='§', engine='python',
                          index_col=0, encoding='utf-8')
df_airline = pd.read_csv('./data/df_airline.csv', sep='§', engine='python',
                         index_col=0, encoding='utf-8')
df_airport = pd.read_csv('./data/df_airport.csv', sep='§', engine='python',
                         index_col=0, encoding='utf-8')
df_country = pd.read_csv('./data/df_country.csv', sep='§', engine='python',
                         index_col=0, encoding='utf-8')


In [0]:
CT_SEP = '_'
CT_HASH = '#'

CT_SUF_B='B'
CT_SUF_I='I'
CT_SUF_E='E'

CT_TAG_STAT = 'STAT'
CT_TAG_MANU = 'MANU'
CT_TAG_AIRP = 'AIRP'
CT_TAG_AIRL = 'AIRL'
CT_TAG_COUN = 'COUN'
CT_TAG_DATE1 = 'DATE1'
CT_TAG_DATE2 = 'DATE2'
CT_TAG_STUD = 'STUDIED'
CT_TAG_O = 'O'

In [8]:
prefixe = [CT_TAG_STAT, CT_TAG_MANU, CT_TAG_AIRP, CT_TAG_AIRL, CT_TAG_COUN,
           CT_TAG_DATE1, CT_TAG_DATE2, CT_TAG_STUD]
separat = [CT_SEP]
suffixe = [CT_SUF_B, CT_SUF_I, CT_SUF_E]

all_tags =  [CT_TAG_O] + ["".join(t) 
                          for t in list(itertools.product(prefixe,separat,suffixe))]
print(all_tags)

['O', 'STAT_B', 'STAT_I', 'STAT_E', 'MANU_B', 'MANU_I', 'MANU_E', 'AIRP_B', 'AIRP_I', 'AIRP_E', 'AIRL_B', 'AIRL_I', 'AIRL_E', 'COUN_B', 'COUN_I', 'COUN_E', 'DATE1_B', 'DATE1_I', 'DATE1_E', 'DATE2_B', 'DATE2_I', 'DATE2_E', 'STUDIED_B', 'STUDIED_I', 'STUDIED_E']


In [0]:
def preprocess(w):
    w = w.split(',')[0].strip()
    w = w.split('(')[0].strip()
    w = w.split('/')[0].strip()
    w = "_".join(list(map(lambda x:x.strip(), w.split('&'))))
    w = w.replace("St ","Saint ")
    w = w.replace("-"," ")
    w = w.replace("."," ")
    w = w.replace("*"," ")
    w = w.replace("\'"," ")
    w = w.split()
    if w[-1].lower() == 'airport': w = w[0:-1]
    if w[-1].lower() == 'intl': w = w[0:-1]
    w = "_".join(w)
    return w

def word_gen(model, word_list, up=False, cap=False, same=False, low=False):
    def my_capitalize(w):
        return "_".join(list(map(lambda x:x.capitalize(), w.split('_'))))
    
    l_upper, l_lower, l_cap, l_same = [], [], [], []
    if up:
        l_upper = [m.upper() for m in word_list 
                   if m.upper() in model.vocab]
    if cap:
        l_cap = [my_capitalize(m) 
                 for m in word_list 
                 if my_capitalize(m) in model.vocab]
    if same:
        l_same = [m for m in word_list 
                  if m in model.vocab]
    if low:
        l_lower = [m.lower() for m in word_list 
                   if m.lower() in model.vocab]
    
    return set(l_upper + l_cap + l_lower + l_same)

In [0]:
# init the tags
def init_sent(list_words):
    res = []
    for e in list_words:
        if e.startswith('#'):
            res.append((e,'#'))
        elif e.startswith('$'):
            res.append((e,'#'))
        else :
            for w in e.split():
                res.append((w,CT_TAG_O))
    return res

# initialize a structure
# if tuple : choose one at random
def init_structure(struct):
    res = []
    if isinstance(struct, str):
        struct = struct.strip()
        if len(struct): res += [struct]
    elif isinstance(struct, tuple):
        n = np.random.choice(len(struct),1)[0]
        l = list(struct)[n]
        res += init_structure(l)
    elif isinstance(struct, list):
        for s in struct:
            if isinstance(s, str):
                s = s.strip()
                if len(s) : res += [s]
            elif isinstance(s,tuple):
                n = np.random.choice(len(s),1)[0]
                l = list(s)[n]
                res += init_structure(l)
            elif isinstance(s,list):
                n = np.random.choice(len(s),1)[0]
                l = list(s)[n]
                res += init_structure(l)
    return res

# make the swaps
def swap(tagger,W):
    def wt_augm(w, tag):
        ws = w.split("_")
        ws = [w for w in ws if w != '']
        t = tag.split(CT_SEP)
        t_name = t[0]
        if len(t)>1:
            t_suffix = t[-1]
        else : t_suffix = CT_SUF_B
        
        tags = [t_name+CT_SEP+t_suffix if i==0 
                else t_name+CT_SEP+CT_SUF_E if i+1==len(ws)
                else t_name+CT_SEP+CT_SUF_I 
                for i in range(len(ws))]
        tags = [CT_TAG_O if t.startswith(CT_TAG_O) 
                else t 
                for t in tags]
        return list(zip(ws,tags))
    
    hw, tag = W
    if hw.startswith('#'):
        n = random.sample(range(len(tagger[hw])),1)[0]
        w = tagger[hw][n]
        w = list(zip(w[0].split(), w[1].split()))
        w = [swap(tagger,(m[0], tag)) if (tag!='#')
             else swap(tagger,(m[0], m[1])) 
             for m in w]
        if len(w)==1: 
            w = w[0]
            
    elif hw.startswith('$GEN$'):
        key = hw.split('$GEN$')[1]
        a = select_from_sentence_gen(gen, key)[1:]
        w = wt_augm(a, tag)
            
    elif hw.startswith('$VOC$'):
        _,key,sub_key = hw.split('$VOC$')[1].split('#')
        words_set = voc[key][sub_key]
        n = random.sample(range(len(words_set)),1)[0]
        w = list(words_set)[n]
        w = wt_augm(w, tag)
    else:
        w = (hw, tag)
    
    return w

# remove list of list
def remove_lists(listsOfLists):
    continue_loop = True
    while continue_loop:
        res = []
        continue_loop = False
        for t in listsOfLists:
            if isinstance(t, list):
                continue_loop = True
                res.extend(t)
            else:
                res.append(t)
        listsOfLists = res
    return res

# generate a new sentence with tags from a structure
def generete_sentence_from_structure(tagger, structure):
    sent_ini = init_sent(structure)
    tagged_sentence = remove_lists([swap(tagger,w) for w in sent_ini])
    sent = " ".join([w[0] for w in tagged_sentence])
    tags = " ".join([w[1] for w in tagged_sentence])
    return sent, tags

# return the closest word in the vocabulary, the key and score
def closest_word_in_voc(voc, word):
    jarowinkler = JaroWinkler()
    sim = 0
    for k in voc:
        for w in voc[k]['voc']:
            jaro = jarowinkler.similarity(w,word)
            if jaro >= sim:
                sim = jaro
                result = w,k,sim
    return result

# autocorrection of a sentence based on the model and our vocabulary
def auto_correction(model_vocab, voc, voc_stopwords, sentence):
    spell = SpellChecker()
    correction = []
    for w in sentence.split():
        if w[0].isupper():
            wc,wk,ws = closest_word_in_voc(voc, w)
            if ws > 0.95:
                correction.append(wc)
            else:
                if w in model_vocab or w in voc_stopwords:
                    correction.append(w)
                else:
                    wc = spell.correction(w)
                    correction.append(wc.capitalize())
        else :
            if w in model_vocab or w in voc_stopwords:
                correction.append(w)
            else:
                wc,wk,ws = closest_word_in_voc(voc, w)
                if ws > 0.95:
                    correction.append(wc)
                else:
                    wc = spell.correction(w)
                    correction.append(wc)
    return correction

# generate and select a sentence from the gen dictionary
def select_from_sentence_gen(gen, ks):
    res = ''
    for k in ks.split():
        if k not in gen:
            print("warning :",k,"not found")
        else :
            n = np.random.choice(len(gen[k]),1)[0]
            l = list(gen[k])[n].split()
            for el in l:

                if el.startswith('#'):
                    res += select_from_sentence_gen(gen, el)
                else :
                    res += '_'+el
    return res

In [0]:
# function that verify the integrity of the tagger dictionary
def verif_dict_tagger(dict_swaps):
    verif_global = True
    hash_keys = [k for k in dict_swaps.keys() if k.startswith('$')==False]
    for k in hash_keys:
        l = dict_swaps[k]
        words = [w[0] for w in l]
        tags = [w[1] for w in l]

        # verif number of words = number of tags
        left = np.array([len(words) 
                         for words in list(map(lambda x:x.split(), words))])
        right = np.array([len(words) 
                          for words in list(map(lambda x:x.split(), tags))])
        verif1 = all(left == right)
        if not verif1: print(k, "\t1-Number of words != number of tags")

        # verif all #words have #tags conterpart
        left = np.array([words[0].startswith("#") 
                         for words in list(map(lambda x:x.split(), words))
                         if len(words)>0])
        right = np.array([words[0].startswith("#") 
                          for words in list(map(lambda x:x.split(), tags))
                          if len(words)>0])
        verif2 = all(left == right)
        if not verif2: print(k, "\t2-Not all words have their tags")
        
        # verif all links have key
        verif3 = [m in dict_swaps.keys() for w in words for m in w.split()
                  if m.startswith("#")]
        if len(verif3)>0: verif3 = all(verif3)
        else: verif3 = True
        if not verif3: print(k, "\t3-Not all link have a key")

        verif_key = verif1 & verif2 & verif3
        verif_global &= verif_key

        if not verif_key: print(k)
    
    return verif_global

# verify the integrity of the generator dictionary
def verif_dict_generator(dict_gen):
    verif_global = True
    
    verif_keys_in_dict = [
        p in gen 
        for k in gen.keys()
        for sent in list(gen[k])
        for p in sent.split()
        if p.startswith('#')
    ]
    
    verif_global &= all(verif_keys_in_dict)
    return verif_global

# verify the integrity of the structures
def verif_structure_link_tag(structure, tagger, display=False):
    words = list(pd.core.common.flatten([[structure]]))
    
    verifs = [w for w in words 
              if w.startswith('#') and w not in tagger.keys()]
    
    if display : print(verifs)
    return len(verifs)==0

# verify the integrity of the tagger in respect of the generator
def verif_dict_tagger_links_gen(tagger,gen):
    verif_global = True
    hash_keys = [k for k in tagger.keys() 
                 if k.startswith('$')==False]
    for k in hash_keys:
        l = tagger[k]
        words = [w[0] for w in l]
        gen_keys = [l.split("$GEN$")[1] for w in words 
                    for l in w.split() 
                    if l.startswith("$GEN$")]
        verif = [gen_k in gen for gen_k in gen_keys]
        verif = all(verif)
        
        verif_global = verif_global & verif
        if not verif: 
            print(k, verif)
        
    return verif_global

# verify the integrity of the tagger in respect of the vocabulary
def verif_dict_tagger_links_voc(tagger,voc):
    verif_global = True
    hash_keys = [k for k in tagger.keys() if k.startswith('$')==False]
    for k in hash_keys:
        l = tagger[k]
        words = [w[0] for w in l]
        gen_keys = [tuple(l.split("$VOC$")[1].split('#')[1:])
                    for w in words for l in w.split() 
                    if l.startswith("$VOC$")]
        
        verif1 = all([gen_k[0] in voc for gen_k in gen_keys])
        verif2 = all([gen_k[1] in voc[gen_k[0]] for gen_k in gen_keys])
        verif = verif1 & verif2
        if not verif: print(k, ": a voc key is not in the voc")
        verif_global = verif_global & verif
        
    return verif_global

In [0]:
manu = ['Airbus', 'Boeing', 'Cessna', 'Cirrus ','Pilatus', 'Matra']
coun = list(df_country['country'].apply(preprocess))
citi = list(df_airport['location'].apply(preprocess))
airp = list(df_airport['airport'].apply(preprocess))
airl = list(df_airline['airline'].apply(preprocess))
mont = ['January', 'February', 'March', 'April', 'May', 'June', 
        'July', 'August', 'September', 'October', 'November', 'December']
seas = ['Winter', 'Spring', 'Autumn', 'Winter']
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 
        'Saturday', 'Sunday']
year = ['Year', 'Years']

In [0]:
voc = {
    "manu" : {
        'voc' : word_gen(model_w2v, manu, up=True, 
                         cap=True, same=True, low=False),
        'tag' : CT_TAG_MANU,
        'name' : 'Airplane Manufacturer'
    },
    "coun" : {
        'voc' : word_gen(model_w2v, coun, up=True, 
                         cap=True, same=True, low=False),
        'tag' : CT_TAG_COUN,
        'name' : 'Country'
    },
    "citi" : {
        'voc' : word_gen(model_w2v, citi, up=True, 
                         cap=True, same=True, low=False),
        'tag' : None,
        'name' : 'City'
    },
    "airp" : {
        'voc' : word_gen(model_w2v, airp, up=True, 
                         cap=True, same=True, low=False),
        'tag' : CT_TAG_AIRP,
        'name' : 'Airport'
    },
    "airl" : {
        'voc' : word_gen(model_w2v, airl, up=True, 
                         cap=True, same=True, low=False),
        'tag' : CT_TAG_AIRL,
        'name' : 'Airline'
    },
    "mont" : {
        'voc' : word_gen(model_w2v, mont, up=True, 
                         cap=True, same=True, low=False),
        'tag' : None,
        'name' : 'Month'
    },
    "seas" : {
        'voc' : word_gen(model_w2v, seas, up=True, 
                         cap=True, same=True, low=True),
        'tag' : None,
        'name' : 'Season'
    },
    "days" : {
        'voc' : word_gen(model_w2v, days, up=True, 
                         cap=True, same=True, low=True),
        'tag' : None,
        'name' : 'Days'
    },
    "year" : {
        #'voc' : word_gen(model_w2v, year, up=True, 
        #cap=True, same=True, low=False),
        'voc' : {str(y) for y in range(1900,2050)},
        'tag' : None,
        'name' : 'Year'
    },
}

In [14]:
gen = {
    '#client' : {
        'client', 'clients', 'customer', 'customers', 'passenger', 'passengers',
    },
    '#satisfaction' : {
        'contentment', 'contentments', 'satisfaction', 'satisfactions',
    },
    '#quantity' : {
        'amount', 'amounts', 'number', 'quantity', 'sum', 'sums',
    },
    '#flight' : {
        'flight', 'flights', 'travel', 'travels',
    },
    '#client_satisfaction' : {
        '#client #satisfaction',
        '#satisfaction of #client'
    },
    '#quantity_of_flights' : {
        '#flight #quantity',
        '#quantity of #flight'
    },
    '#PROP#to' : {
        'to', 'to the beginning of', 'to the end of', 'until',
    },
    '#PROP#from' : {
        'from', 'from the beginning of', 'from the end of', 'in',
    },
    '#STATS#graph' : {
        'chart','charts', 'graph','graphs', 'histogram', 'histograms',
        'pie chart', 'pie charts', 'slope', 'slopes',
    },
    '#STATS#wordcloud' : {
        'cloud of words', 'wordcloud',
    },
    '#STATS#plan_of_plane' : {
        'plan of plane', 'plans of plane', 'plans of planes',
    },
    '#VERB#show' : {
        'display', 'highlight', 'plot', 'print', 'show', 'view',
    },
    '#VERB#be' : {
        'happen to be', 'is', 'seems', 'seems to be', 
    },
    '#VERB#like' : {
        'like', 'likes', 'love', 'loves',
    },
    '#VERB#dislike' : {
        'cannot stand', 'dislike', 'dislikes', 'hate', 'hates',
    },
    '#PRONON#meus' : {
        'me', 'us',
    },
    '#ARTICLE#' : {
        'a', 'an', 'the',
    },
    '#COMP#good' : {
        'able', 'acceptable', 'ace', 'admirable', 'advantageous', 'agreeable',
        'amazing', 'appropriate', 'awesome', 'benefic', 'capable', 'capital', 
        'clever', 'comfortable', 'commendable', 'common', 'congenial', 
        'convenient', 'decent', 'deluxe', 'efficient', 'excellent', 
        'exceptional', 'expert', 'fascinating', 'favorable', 'first-class', 
        'first-rate', 'flawless', 'fresh', 'friendly', 'good', 'gratifying', 
        'great', 'healthy', 'helpful', 'honest', 'honorable', 'hygienic', 
        'incredible', 'intact', 'kindhearted', 'marvelous', 'neat', 'nice', 
        'normal', 'opportune', 'perfect', 'pleasant', 'pleasing', 'positive', 
        'precious', 'prime', 'prodigious', 'profitable', 'qualified', 'rad', 
        'reliable', 'reputable', 'respectable', 'right', 'safe', 'salutary',
        'satisfactory', 'satisfying', 'serviceable', 'shipshape', 'shocking', 
        'skillful','solid', 'splendid', 'stable', 'sterling', 'stunning', 
        'stupendous', 'suitable', 'suited', 'super', 'superb', 'superior', 
        'surprising', 'talented', 'tasty', 'tip-top', 'tolerable', 
        'trustworthy', 'unbelievable', 'useful', 'valuable', 'welcome', 
        'wonderful', 'worthy',
    },
    '#COMP#bad' : {
        'abominable', 'amiss', 'atrocious', 'awful', 'bad', 'bummer', 'careless', 
        'catastrophic', 'chaotic', 'cheap', 'cheesy', 'crap', 'crappy', 'crummy', 
        'damaging', 'dangerous', 'defective', 'deficient', 'deleterious', 
        'detrimental', 'disagreeable', 'disastrous', 'discouraging', 'displeasing', 
        'distressing', 'dreadful', 'dumb', 'erroneous', 'evil', 'fallacious', 
        'garbage', 'godawful', 'grim', 'grody', 'gross', 'grungy', 'harsh', 'hurtful', 
        'icky', 'imperfect', 'impolite', 'inadequate', 'incorrect', 'iniquitous', 
        'injurious', 'junky', 'lame', 'loud', 'lousy', 'mean', 'moldy', 'noisy', 
        'not good', 'old', 'painful', 'poor', 'rancid', 'regretful', 'rotten', 
        'rude', 'ruinous', 'sad', 'shitty', 'slipshod', 'spoiled', 
        'stinking', 'strident', 'substandard', 'terrible', 'tragic', 'troubled', 
        'troubling', 'unacceptable', 'unfavorable', 'unfortunate', 'unhappy',
        'unhealthy', 'unlucky', 'unpleasant', 'unsatisfactory', 'unwell', 'upsetting', 
        'vicious', 'wicked', 'wrong',
    },
}

verif_dict_generator(gen)

True

In [15]:
select_from_sentence_gen(gen, "#COMP#good #client_satisfaction")[1:]

'honest_contentment_of_clients'

In [16]:
tagger = {
    '#STATS#graph' : [('$GEN$#STATS#graph', CT_TAG_STAT)],
    '#STATS#wordcloud' : [('$GEN$#STATS#wordcloud', CT_TAG_STAT)],
    '#STATS#plan_of_plane' : [('$GEN$#STATS#plan_of_plane', CT_TAG_STAT)],
    '#VERB#show' : [('$GEN$#VERB#show', CT_TAG_O)],
    '#PRONON#meus' : [('$GEN$#PRONON#meus', CT_TAG_O)],
    '#ARTICLE#' : [('$GEN$#ARTICLE#', CT_TAG_O)],

    # Named Entity
    '#NE#manu' : [
        ('$VOC$#manu#voc', CT_TAG_MANU),
    ],

    '#NE#airp' : [
        ('$VOC$#airp#voc', CT_TAG_AIRP),
        ('$VOC$#airp#voc airport', CT_TAG_AIRP+' '+CT_TAG_O),
        ('airport of $VOC$#airp#voc', CT_TAG_O+' '+CT_TAG_O+' '+CT_TAG_AIRP),
    ],

    '#NE#airl' : [
        ('$VOC$#airl#voc', CT_TAG_AIRL),
    ],

    '#NE#coun' : [
        ('$VOC$#coun#voc', CT_TAG_COUN),
    ],

    # Date1
    '#DATE1#' : [
      ('#DATE1#y', CT_HASH),
      ('year #DATE1#y', CT_TAG_O+' '+CT_HASH),
      ('#DATE1#my', CT_HASH),
      ('#DATE1#sy', CT_HASH),
    ],
    '#DATE1#y' : [
      ('$VOC$#year#voc',
       CT_TAG_DATE1+CT_SEP+CT_SUF_B),
    ],
    '#DATE1#my' : [
      ('$VOC$#mont#voc $VOC$#year#voc',
       CT_TAG_DATE1+CT_SEP+CT_SUF_B+' '+\
       CT_TAG_DATE1+CT_SEP+CT_SUF_E),
    ],
    '#DATE1#sy' : [
      ('$VOC$#seas#voc $VOC$#year#voc',
       CT_TAG_DATE1+CT_SEP+CT_SUF_B+' '+\
       CT_TAG_DATE1+CT_SEP+CT_SUF_E),
    ],

    # Date2
    '#DATE2#' : [
      ('#DATE2#y', CT_HASH),
      ('year #DATE2#y', CT_TAG_O+' '+CT_HASH),
      ('#DATE2#my', CT_HASH),
      ('#DATE2#sy', CT_HASH),
    ],
    '#DATE2#y' : [
      ('$VOC$#year#voc',
       CT_TAG_DATE2+CT_SEP+CT_SUF_B),
    ],
    '#DATE2#my' : [
      ('$VOC$#mont#voc $VOC$#year#voc',
       CT_TAG_DATE2+CT_SEP+CT_SUF_B+' '+\
       CT_TAG_DATE2+CT_SEP+CT_SUF_E),
    ],
    '#DATE2#sy' : [
      ('$VOC$#seas#voc $VOC$#year#voc',
       CT_TAG_DATE2+CT_SEP+CT_SUF_B+' '+\
       CT_TAG_DATE2+CT_SEP+CT_SUF_E),
    ],

    # Studied variable
    '#STUDIED#' : [
      ('#ARTICLE# $GEN$#quantity_of_flights', CT_HASH+' '+CT_TAG_STUD),
      ('#ARTICLE# $GEN$#client_satisfaction', CT_HASH+' '+CT_TAG_STUD),
    ],

    # Propositions SHOW
    '#PROP#show_meus_the' : [
      ('#VERB#show #PRONON#meus #ARTICLE#', CT_HASH+' '+CT_HASH+' '+CT_HASH),
    ],

    # Propositions DATES
    '#PROP#DATE#' : [
        ('#PROP#DATE#from_to', CT_HASH),
        ('#PROP#DATE#for', CT_HASH),
        ('#PROP#DATE#since', CT_HASH),
    ],
    '#PROP#DATE#from_to' : [
      ('$GEN$#PROP#from #DATE1# $GEN$#PROP#to #DATE2#',
       CT_TAG_O+' '+CT_HASH+' '+CT_TAG_O+' '+CT_HASH),
    ],
    '#PROP#DATE#for' : [
        ('for #DATE1#', CT_TAG_O+' '+CT_HASH),
        ('for the #DATE1#', CT_TAG_O+' '+CT_TAG_O+' '+CT_HASH),
    ],
    '#PROP#DATE#since' : [
        ('since #DATE1#', CT_TAG_O+' '+CT_HASH),
        ('since the #DATE1#', CT_TAG_O+' '+CT_TAG_O+' '+CT_HASH),
    ],
}

print(verif_dict_tagger(tagger))
print(verif_dict_tagger_links_gen(tagger,gen))
print(verif_dict_tagger_links_voc(tagger,voc))

True
True
True


In [36]:
s_do_or_not = ('', '#PROP#show_meus_the',)
s_what = ('#STATS#plan_of_plane',
          '#STATS#wordcloud',
          ['#STATS#graph', 'of', '#STUDIED#'],
         )
s_who_or_not = ('',
                ['for', ('#NE#manu',['#NE#manu', ('and',',',''), '#NE#manu']),],
               )
s_when_or_not = ('',
                 ['in',('#NE#coun',['#NE#coun',('and',',',''),'#NE#coun']),"#PROP#DATE#",],
                )

structures = [
    [s_do_or_not, s_what, s_who_or_not, s_when_or_not,],
    [s_do_or_not, s_what, s_when_or_not, s_who_or_not,],
    [s_who_or_not, s_do_or_not, s_what, s_when_or_not,],
    [s_when_or_not, s_do_or_not, s_what, s_who_or_not,],
    [s_who_or_not, s_when_or_not, s_do_or_not, s_what,],
    [s_when_or_not, s_who_or_not, s_do_or_not, s_what,],
]

verif_structure_link_tag(structures, tagger, display=False)

True

In [37]:
structure = structures[np.random.choice(len(structures),1)[0]]
init_structure(structure)

['for',
 '#NE#manu',
 ',',
 '#NE#manu',
 '#PROP#show_meus_the',
 '#STATS#graph',
 'of',
 '#STUDIED#']

In [38]:
structure = structures[np.random.choice(len(structures),1)[0]]
structure_init = init_structure(structure)
sent,tags = generete_sentence_from_structure(tagger,structure_init)
print("Same length :", len(sent.split()) == len(tags.split()))
sent,tags

Same length : True


('display us a plans of plane for BOEING',
 'O O O STAT_B STAT_I STAT_E O MANU_B')

In [39]:
sentence = 'I woulde lirke the graph of the numbr of fligt for Boing on mondy 2020'
auto_correction(model_w2v.vocab, voc, voc_stopwords, sentence)

['I',
 'would',
 'like',
 'the',
 'graph',
 'of',
 'the',
 'number',
 'of',
 'flight',
 'for',
 'Boeing',
 'on',
 'monday',
 '2020']

**TRAINING**

In [21]:
pip install simpletransformers

     |████████████████████████████████| 102kB 2.4MB/s 
     |████████████████████████████████| 1.4MB 8.7MB/s 
     |████████████████████████████████| 204kB 51.6MB/s 
     |████████████████████████████████| 450kB 48.1MB/s 
     |████████████████████████████████| 92kB 12.9MB/s 
     |████████████████████████████████| 102kB 12.8MB/s 
     |████████████████████████████████| 460kB 48.0MB/s 
     |████████████████████████████████| 102kB 12.5MB/s 
     |████████████████████████████████| 870kB 39.4MB/s 
     |████████████████████████████████| 1.0MB 40.1MB/s 
     |████████████████████████████████| 71kB 10.0MB/s 
     |████████████████████████████████| 71kB 9.2MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=bdd2f746656918e2a456d1405d4ae2dcedb957db7c8bbc388e11319c5ad1a59f
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
  Created wheel for shortuuid: filename=shortuuid-0.5.0-cp36-none-any.whl 

In [22]:
from simpletransformers.ner import NERModel

In [0]:
data_train = []
for i in range(50000):
  structure = structures[np.random.choice(len(structures),1)[0]]
  structure_init = init_structure(structure)
  sent,tags = generete_sentence_from_structure(tagger,structure_init)
  for u,j in zip(sent.split(),tags.split()):
    data_train.append([i,u,j])

In [0]:
data_eval = []
for i in range(1000):
  structure = structures[np.random.choice(len(structures),1)[0]]
  structure_init = init_structure(structure)
  sent,tags = generete_sentence_from_structure(tagger,structure_init)
  for u,j in zip(sent.split(),tags.split()):
    data_eval.append([i,u,j])

In [0]:
# Creating train_df  and eval_df 
train_df = pd.DataFrame(data_train, columns=['sentence_id', 'words', 'labels'])
eval_df = pd.DataFrame(data_eval, columns=['sentence_id', 'words', 'labels'])

In [0]:
# Create a NERModel
model = NERModel('bert', 'bert-base-cased', 
                 args={'overwrite_output_dir': True,
                       'reprocess_input_data': True},
                 use_cuda = False,
                 labels = all_tags)

In [47]:
# Train the model
model.train_model(train_df)

Converting to features started.


Running loss: 0.000290

KeyboardInterrupt: ignored

In [48]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

Converting to features started.


KeyboardInterrupt: ignored

In [49]:
sentence = 'I woulde lirke the graph of the numbr of fligt for Boing on mondy 2020'
corrected = " ".join(auto_correction(model_w2v.vocab, voc, voc_stopwords, sentence))
corrected

'I would like the graph of the number of flight for Boeing on monday 2020'

In [50]:
predictions, raw_outputs = model.predict([corrected])
print(predictions)

Converting to features started.


[[{'I': 'O'}, {'would': 'O'}, {'like': 'O'}, {'the': 'O'}, {'graph': 'STAT_B'}, {'of': 'O'}, {'the': 'O'}, {'number': 'STUDIED_B'}, {'of': 'STUDIED_I'}, {'flight': 'STUDIED_E'}, {'for': 'O'}, {'Boeing': 'MANU_B'}, {'on': 'O'}, {'monday': 'DATE1_B'}, {'2020': 'DATE1_E'}]]


In [51]:
eval_df

,sentence_id,words,labels
0,0,in,O
1,0,Iran,COUN_B
2,0,and,O
3,0,Mozambique,COUN_B
4,0,since,O
...,...,...,...
53354,4999,planes,STAT_E
53355,4999,in,O
53356,4999,LATVIA,COUN_B
53357,4999,since,O


In [52]:
resultats = {}
for i in eval_df['sentence_id'].unique()[0:10]:
  sentence = " ".join(list(eval_df.loc[eval_df['sentence_id'] == i, 'words']))
  predictions, raw_outputs = model.predict([sentence])
  resultats[i] = predictions

Converting to features started.


Converting to features started.


Converting to features started.


Converting to features started.


Converting to features started.


Converting to features started.


Converting to features started.


Converting to features started.


Converting to features started.


Converting to features started.


In [53]:
resultats

{0: [[{'in': 'O'},
   {'Iran': 'COUN_B'},
   {'and': 'O'},
   {'Mozambique': 'COUN_B'},
   {'since': 'O'},
   {'the': 'O'},
   {'SPRING': 'DATE1_B'},
   {'1931': 'DATE1_E'},
   {'show': 'O'},
   {'me': 'O'},
   {'an': 'O'},
   {'cloud': 'STAT_B'},
   {'of': 'STAT_I'},
   {'words': 'STAT_E'}]],
 1: [[{'for': 'O'},
   {'Airbus': 'MANU_B'},
   {'Boeing': 'MANU_B'},
   {'cloud': 'STAT_B'},
   {'of': 'STAT_I'},
   {'words': 'STAT_E'}]],
 2: [[{'plans': 'STAT_B'}, {'of': 'STAT_I'}, {'planes': 'STAT_E'}]],
 3: [[{'pie': 'STAT_B'},
   {'chart': 'STAT_E'},
   {'of': 'O'},
   {'the': 'O'},
   {'number': 'STUDIED_B'},
   {'of': 'STUDIED_I'},
   {'flight': 'STUDIED_E'},
   {'in': 'O'},
   {'NETHERLANDS': 'COUN_B'},
   {'ANTILLES': 'COUN_E'},
   {'for': 'O'},
   {'year': 'O'},
   {'1940': 'DATE1_B'}]],
 4: [[{'plans': 'STAT_B'}, {'of': 'STAT_I'}, {'plane': 'STAT_E'}]],
 5: [[{'plot': 'O'},
   {'us': 'O'},
   {'the': 'O'},
   {'plans': 'STAT_B'},
   {'of': 'STAT_I'},
   {'plane': 'STAT_E'},
   {'for